# Loadings

In [ ]:
!pip install selenium
!pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 17.1 MB/s eta 0:00:00


In [ ]:
import google_colab_selenium as gs
import pandas as pd
import requests
from collections import Counter

# User Input

In [ ]:
# User inputs and Parameters
inputs = input("Enter Your Future Commander: ")
input_end = inputs.replace(" ", "_")
input_card = inputs.replace(" ","+").replace(",","%2C")

# Variable for putting counts of cards in output file
show_counts = False
# Shows details in final code cell
show_details = True

Enter Your Future Commander: Slinza, the Spiked Stampede


# Webscraping and Downloads

In [ ]:
# Webscraping MTG Goldfish
driver = gs.Chrome()
deck_links = []
deck_names = []
deck_details_raw = []

try:
    url = f"https://www.mtggoldfish.com/deck_searches/create?utf8=%E2%9C%93&deck_search%5Bname%5D=&deck_search%5Bformat%5D=commander&deck_search%5Btypes%5D%5B%5D=&deck_search%5Btypes%5D%5B%5D=tournament&deck_search%5Btypes%5D%5B%5D=user&deck_search%5Bplayer%5D=&deck_search%5Bdate_range%5D=09%2F20%2F2024+-+12%2F20%2F2024&deck_search%5Bdeck_search_card_filters_attributes%5D%5B0%5D%5Bcard%5D={input_card}&deck_search%5Bdeck_search_card_filters_attributes%5D%5B0%5D%5Bquantity%5D=1&deck_search%5Bdeck_search_card_filters_attributes%5D%5B0%5D%5Btype%5D=commander&deck_search%5Bdeck_search_card_filters_attributes%5D%5B1%5D%5Bcard%5D=&deck_search%5Bdeck_search_card_filters_attributes%5D%5B1%5D%5Bquantity%5D=1&deck_search%5Bdeck_search_card_filters_attributes%5D%5B1%5D%5Btype%5D=maindeck&counter=2&commit=Search"
    driver.get(url)

    # Fetch links and deck names from the search results, change 10 to however many desired decks
    for i in range(10):
        link_element = driver.find_element("xpath", value=f'/html/body/main/div/div[3]/table/tbody/tr[{i+1}]/td[2]/a')
        deck_links.append(link_element.get_attribute('href'))
        deck_names.append(link_element.text)

    # Visit each deck link and fetch additional details
    for deck_link in deck_links:
        driver.get(deck_link)
        details = driver.find_elements("xpath", value="//tr[@class='deck-category-header']")
        deck_details_raw.append([detail.text for detail in details])  # Collect text of deck details

    # Print results
    print("Deck Names:", deck_names)
    print("Deck Links:", deck_links)
    for i, details in enumerate(deck_details_raw):
        print(f"Details for Deck {i + 1}:", details)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    driver.quit()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Deck Names: ['Beastmode', 'Slinza', 'Slinza', 'Releasing the Beasts', 'Roid Raging Beasties', 'Slina', 'Slinza, the Spiked Stampede', "Slinza's Wildpack", 'Beastie Boys', 'Slinza Beasts']
Deck Links: ['https://www.mtggoldfish.com/deck/6814116', 'https://www.mtggoldfish.com/deck/6814924', 'https://www.mtggoldfish.com/deck/6808084', 'https://www.mtggoldfish.com/deck/6806184', 'https://www.mtggoldfish.com/deck/6798312', 'https://www.mtggoldfish.com/deck/6796391', 'https://www.mtggoldfish.com/deck/6797528', 'https://www.mtggoldfish.com/deck/6795348', 'https://www.mtggoldfish.com/deck/6793032', 'https://www.mtggoldfish.com/deck/6793169']
Details for Deck 1: ['Commander', 'Creatures (35)', 'Planeswalkers (2)', 'Spells (16)', 'Artifacts (5)', 'Enchantments (9)', 'Lands (32)', '100 Cards Total', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Details for Deck 2: ['Commander', 'Creatures (34)', 'Spells (17)', 'Artifacts (6)', 'Enchantments (9)', 'Lands (33) +2 MDFCs', '100 Cards

In [ ]:
# Remove Empty Entries
for i in deck_details_raw:
  deck_details_raw[deck_details_raw.index(i)] = [x for x in i if x != '']

In [ ]:
# String Manipulation for Download Links
download_links = []
for i in range(len(deck_links)):
  linker = deck_links[i].split('/')
  texter = ''
  texter = linker[0] + '//' + linker[2] + '/' + linker[3] + '/download' + '/' + linker[4]
  download_links.append(texter)

In [ ]:
# Create dataframe for details
columns = ['Deck Name', 'Creatures', 'Planeswalkers', 'Spells', 'Artifacts', 'Enchantments', 'Others', 'Lands', 'Total Cards']
data = []

for name, details in zip(deck_names, deck_details_raw):
    deck_info = {col: 0 for col in columns}
    deck_info['Deck Name'] = name

    for detail in details:
        if 'Creatures' in detail:
            deck_info['Creatures'] = int(detail.split('(')[1].split(')')[0])
        elif 'Planeswalkers' in detail:
            deck_info['Planeswalkers'] = int(detail.split('(')[1].split(')')[0])
        elif 'Spells' in detail:
            deck_info['Spells'] = int(detail.split('(')[1].split(')')[0])
        elif 'Artifacts' in detail:
            deck_info['Artifacts'] = int(detail.split('(')[1].split(')')[0])
        elif 'Enchantments' in detail:
            deck_info['Enchantments'] = int(detail.split('(')[1].split(')')[0])
        elif 'Others' in detail:
            deck_info['Others'] = int(detail.split('(')[1].split(')')[0])
        elif 'Lands' in detail:
            lands_part = detail.split('(')[1].split(')')[0].split('+')
            deck_info['Lands'] = int(lands_part[0])
        elif 'Cards Total' in detail:
            deck_info['Total Cards'] = 100

    data.append(deck_info)

df = pd.DataFrame(data, columns=columns).fillna(0)
numeric_columns = ['Creatures', 'Planeswalkers', 'Spells', 'Artifacts', 'Enchantments', 'Others', 'Lands', 'Total Cards']
df[numeric_columns] = df[numeric_columns].astype(int)

In [ ]:
# Loop through each URL and download the file
filenames = []
for idx, url in enumerate(download_links):
    try:
        print(f"Downloading file {idx + 1}/{len(download_links)}: {url}")
        response = requests.get(url)
        response.raise_for_status()

        filename = url.split("/")[-1]
        filenames.append(filename)

        with open(f'{filename}.txt', "wb") as file:
            file.write(response.content)

        print(f"File saved as: {filename}.txt")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

print("Download complete!")

File saved as: 6814116.txt
File saved as: 6814924.txt
File saved as: 6808084.txt
File saved as: 6806184.txt
File saved as: 6798312.txt
File saved as: 6796391.txt
File saved as: 6797528.txt
File saved as: 6795348.txt
File saved as: 6793032.txt
File saved as: 6793169.txt
Download complete!


# Unify Files and Remove Dupes

In [ ]:
# Unique Card Names From Files
def get_unique_card_names(filenames, show_counts=False):
    card_counter = Counter()

    for file_path in filenames:
        try:
            with open(f"{file_path}.txt", 'r') as file:
                for line in file:
                    # Split each line to remove the quantity at the beginning
                    card_name = ' '.join(line.strip().split(' ')[1:])
                    if card_name:  # Avoid adding empty entries
                        card_counter[card_name] += 1
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")

    return card_counter if show_counts else set(card_counter.keys())

# Combine Unique Names
def compile_unique_card_names(unique_names, show_counts=False):
    if show_counts:
        return [f"{count} {name}" for name, count in sorted(unique_names.items())]
    else:
        return [f"1 {name}" for name in sorted(unique_names)]

# Write to files
def write_to_file(formatted_names, output_file):
    try:
        with open(output_file, 'w') as file:
            file.write("\n".join(formatted_names))
        print(f"Formatted names written to {output_file}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

# Execution
unique_names = get_unique_card_names(filenames, show_counts)
formatted_names = compile_unique_card_names(unique_names, show_counts)
write_to_file(formatted_names, f"{input_end}.txt")


Formatted names written to Slinza,_the_Spiked_Stampede.txt


# Results

In [ ]:
# Show Details
if show_details == True:
    print("Deck Names:", deck_names)
    print("Deck Links:", deck_links)

print(f"File saved as '{input_end}.txt'")
df

Deck Names: ['Beastmode', 'Slinza', 'Slinza', 'Releasing the Beasts', 'Roid Raging Beasties', 'Slina', 'Slinza, the Spiked Stampede', "Slinza's Wildpack", 'Beastie Boys', 'Slinza Beasts']
Deck Links: ['https://www.mtggoldfish.com/deck/6814116', 'https://www.mtggoldfish.com/deck/6814924', 'https://www.mtggoldfish.com/deck/6808084', 'https://www.mtggoldfish.com/deck/6806184', 'https://www.mtggoldfish.com/deck/6798312', 'https://www.mtggoldfish.com/deck/6796391', 'https://www.mtggoldfish.com/deck/6797528', 'https://www.mtggoldfish.com/deck/6795348', 'https://www.mtggoldfish.com/deck/6793032', 'https://www.mtggoldfish.com/deck/6793169']
File saved as 'Slinza,_the_Spiked_Stampede.txt'


,Deck Name,Creatures,Planeswalkers,Spells,Artifacts,Enchantments,Others,Lands,Total Cards
0,Beastmode,35,2,16,5,9,0,32,100
1,Slinza,34,0,17,6,9,0,33,100
2,Slinza,35,2,16,5,9,0,32,100
3,Releasing the Beasts,40,0,11,3,5,0,40,100
4,Roid Raging Beasties,34,0,16,6,9,0,34,100
5,Slina,37,1,17,5,4,0,35,100
6,"Slinza, the Spiked Stampede",29,0,19,6,9,0,36,100
7,Slinza's Wildpack,38,2,17,1,7,0,34,100
8,Beastie Boys,33,3,20,1,5,0,37,100
9,Slinza Beasts,37,0,17,1,8,0,36,100
